### Instalation 
To run this notebook you should prepare your virtual env like conda or env. This code works on
- Python 3.9*
- opencv-python
- nibabel
- matplotlib
- scikit-image
- scipy
- surface-distance
- pycimage (This library don't work on mac but don't worry, you can plot images in other way)

In [1]:
import cv2
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage import measure
from scipy import ndimage
from scipy.ndimage import generate_binary_structure
from surface_distance import metrics

try:
    from pycimg import CImg
except ImportError:
    print("CImg not found, some functions may not work")
    pass

CImg not found, some functions may not work


In [2]:
def show_slices(I, use_cimg=False):
    def show_slices_with_plt():
        fig_rows = 4
        fig_cols = 4
        n_subplots = fig_rows * fig_cols
        n_slice = I.shape[2]
        step_size = n_slice // n_subplots
        plot_range = n_subplots * step_size
        start_stop = int((n_slice - plot_range) / 2)

        fig, axs = plt.subplots(fig_rows, fig_cols, figsize=[10, 10])
        for idx, _img in enumerate(range(start_stop, plot_range, step_size)):
            axs.flat[idx].imshow(I[:, :, _img], cmap='gray')
            axs.flat[idx].axis('off')

        plt.tight_layout()
        plt.show()
        
    if use_cimg:
        try:
            CImg(I).display()
        except:
            show_slices_with_plt()
    else:
        show_slices_with_plt()

In [3]:
class Loader:
    def __init__(self, base_path='.'):
        self.base_path = base_path


    def get_bodymask(self, idx: int):

        return nib.load(f"{self.base_path}/BodyMasks/BODYMASK_IMG_{str(idx).zfill(4)}.nii.gz").get_fdata()

    def get_image(self, idx: int):
        return nib.load(f"{self.base_path}/Images/IMG_{str(idx).zfill(4)}.nii.gz").get_fdata()

    def get_reference(self, idx: int):
        return nib.load(f"{self.base_path}/ReferenceSegmentations/LUNGS_IMG_{str(idx).zfill(4)}.nii.gz").get_fdata()

In [4]:
class Segmenter:
    def __init__(self, img_idx: int):
        self.loader = Loader()
        self.org_img = self.loader.get_image(img_idx)
        self.img = np.where(self.org_img >= -320, 0, 1).astype(np.uint8)
        self.bodymask = self.make_bodymask()
        self.lung_mask = None
        self.labels = None
        self.left_lung = None
        self.right_lung = None
        self.reference_bodymask = self.loader.get_bodymask(img_idx).astype(bool)
        self.reference_img = self.loader.get_reference(img_idx)
        self.reference_left_lung = None
        self.reference_right_lung = None
    
    def make_bodymask(self):
        body_mask = np.zeros_like(self.img, dtype=np.uint8)
        binary_mask = cv2.bitwise_not(self.img).astype(np.uint8)

        elem = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

        for _i in range(0, body_mask.shape[2]):
            bm = binary_mask[:, :, _i]
            bm = cv2.erode(bm, elem, iterations=1)
            bm = cv2.dilate(bm, elem, iterations=1)
            cv2.floodFill(bm, None, (0, 0), 0)
            cv2.floodFill(bm, None, (0, binary_mask.shape[0]-1), 0)
            
            numLabels, labels, stats, centroids = cv2.connectedComponentsWithStats(bm)
            
            if numLabels > 1:
                max_label = np.argmax(stats[1:, cv2.CC_STAT_AREA]) + 1
                bm = np.where(labels == max_label, 1, 0)
                
            body_mask[:, :, _i] = bm
        
        return body_mask.astype(bool)
    
    def prepare_lungs_mask(self):
        _img  = self.img.copy()
        _bodymask = self.bodymask.copy()
        _img = _img * _bodymask
        struct_elem = generate_binary_structure(3, 1)
        closed = ndimage.binary_closing(_img, structure=struct_elem, iterations=2)
        opened = ndimage.binary_opening(closed, structure=struct_elem, iterations=2)
        filled = ndimage.binary_fill_holes(opened)

        _labels, _ = measure.label(filled, return_num=True)
        
        props = measure.regionprops(_labels)
        
        VOLUME_THRESHOLD = 20000  # Minimum volume in voxels to be considered as lungs
        
        _lung_mask = np.zeros_like(_labels, dtype=np.uint8)
        for prop in props:
            if prop.area >= VOLUME_THRESHOLD:  # Keep only large regions (e.g. lungs)
                _lung_mask[_labels == prop.label] = 1
                
        # Remove the rest of small objects
        _mask = _lung_mask.copy()

        _lung_mask = cv2.erode(_lung_mask, np.ones((9, 9), np.uint8), iterations=1)
        _lung_mask = cv2.dilate(_lung_mask, np.ones((9, 9), np.uint8), iterations=3)
        _lung_mask = _mask & _lung_mask
                
        self.lung_mask = _lung_mask
    
    def remove_trahea(self):
        _lung_mask2 = self.lung_mask.copy()
        
        def reconstruct_frame(_mask, _frame):
            new_mask = _mask & _frame
            next_mask = cv2.dilate(new_mask, np.ones((3,3), np.uint8), iterations=1) & _frame
            while not np.array_equal(new_mask, next_mask):
                new_mask = next_mask
                next_mask = cv2.dilate(new_mask, np.ones((3,3), np.uint8), iterations=1) & _frame
                if np.sum(next_mask) > np.sum(_mask & _frame)*10:
                    return _mask & _frame
            return new_mask
        
        def find_mask_and_first_idx(_mask):
            for i in range(_mask.shape[2]-1, -1, -1):
                if np.sum(_mask[:,:,i]) > 0:
                    return _mask[:,:,i], i
            return None, None

        trahea_mask = np.zeros_like(_lung_mask2)
        mask, trahea_idx = find_mask_and_first_idx(_lung_mask2)
        for _i in range(trahea_idx, -1, -1):
            frame = _lung_mask2[:,:,_i]
            mask = reconstruct_frame(mask, frame)
            trahea_mask[:,:,_i] = mask

        self.lung_mask = _lung_mask2 - trahea_mask

    
    def label_lungs(self):
        distances = ndimage.distance_transform_edt(self.lung_mask)
        
        distances_left = distances.copy()
        distances_left[:, 256:, :] = 0
        
        distances_right = distances.copy()
        distances_right[:, :256, :] = 0
        
        coord_left = peak_local_max(distances_left, num_peaks=1)[0]
        coord_right = peak_local_max(distances_right, num_peaks=1)[0]
        
        markers = np.zeros_like(self.lung_mask, dtype=np.uint8)
        markers[coord_left[0], coord_left[1], coord_left[2]] = 1
        markers[coord_right[0], coord_right[1], coord_right[2]] = 2

        self.labels = watershed(-distances, markers, mask=self.lung_mask)
    
    def divide_lungs(self):
        right_idx = np.unravel_index(np.argmax(np.where(self.labels == 1, 1, 0)), self.labels.shape)[1]
        left_idx = np.unravel_index(np.argmax(np.where(self.labels == 2, 1, 0)), self.labels.shape)[1]
        
        _right_lung = np.where(self.labels == 1, 1, 0).astype(bool)
        _left_lung = np.where(self.labels == 2, 1, 0).astype(bool)
        
        self.left_lung, self.right_lung = (_left_lung, _right_lung) if right_idx > left_idx else (_right_lung, _left_lung)

    def divide_segmentation(self):
        """
            Divide the reference segmentation into left and right lung
        """
        _left_lung = np.where(self.reference_img == 2, 1, 0).astype(bool)
        _right_lung = np.where(self.reference_img == 3, 1, 0).astype(bool)
        self.reference_left_lung, self.reference_right_lung = (_left_lung, _right_lung)

In [5]:
class Metrics:
    @staticmethod
    def compute_dice(result, _reference):
        return metrics.compute_dice_coefficient(result, _reference)

    @staticmethod
    def compute_housdorf(segmentation_result, reference_segmentation, voxel_spacing = (1, 1, 1)):
        surface_distances = metrics.compute_surface_distances(
            reference_segmentation,
            segmentation_result,
            spacing_mm=voxel_spacing
            )
        hausdorff_distance = metrics.compute_robust_hausdorff(
            surface_distances,
            percent=100
            )
        return hausdorff_distance

    @staticmethod
    def compute_metrics(segmentation_result, reference_segmentation, voxel_spacing = (1, 1, 1)):
        dice = Metrics.compute_dice(segmentation_result, reference_segmentation)
        hausdorff = Metrics.compute_housdorf(segmentation_result, reference_segmentation, voxel_spacing)
        return dice, hausdorff

# Test accuracy on images

In [6]:
for i in [1, 7, 10, 11, 12, 13, 14]:
    segmenter = Segmenter(i)
    segmenter.prepare_lungs_mask()
    segmenter.remove_trahea()
    segmenter.label_lungs()
    segmenter.divide_lungs()
    segmenter.divide_segmentation()

    left_dice, left_housdorf = Metrics.compute_metrics(segmenter.left_lung, segmenter.reference_left_lung)
    right_dice, right_housdorf = Metrics.compute_metrics(segmenter.right_lung, segmenter.reference_right_lung)
    bodymask_dice, bodymask_housdorf = Metrics.compute_metrics(segmenter.bodymask, segmenter.reference_bodymask)
    
    print(f"Image dice: {i}: Left lung: {left_dice}, Right lung: {right_dice}, Bodymask: {bodymask_dice}")    
    print(f"Image housdorf: {i}: Left lung: {left_housdorf}, Right lung: {right_housdorf}, Bodymask: {bodymask_housdorf}")

Image dice: 1: Left lung: 0.9842289238459231, Right lung: 0.9867282348039378, Bodymask: 0.9974271433421434
Image housdorf: 1: Left lung: 17.832554500127006, Right lung: 16.401219466856727, Bodymask: 26.076809620810597
Image dice: 7: Left lung: 0.9846273752537384, Right lung: 0.9751486850350803, Bodymask: 0.9985053543234377
Image housdorf: 7: Left lung: 32.71085446759225, Right lung: 31.272991542223778, Bodymask: 20.0
Image dice: 10: Left lung: 0.9874051320075206, Right lung: 0.986278954267094, Bodymask: 0.9973139561886235
Image housdorf: 10: Left lung: 21.95449840010015, Right lung: 23.345235059857504, Bodymask: 9.0
Image dice: 11: Left lung: 0.9802308407746525, Right lung: 0.9187749854050722, Bodymask: 0.9971283613561217
Image housdorf: 11: Left lung: 39.761790704142086, Right lung: 34.14674215792775, Bodymask: 9.486832980505138
Image dice: 12: Left lung: 0.9863789670462626, Right lung: 0.9887985807935349, Bodymask: 0.9984240218191143
Image housdorf: 12: Left lung: 31.54362059117501, 

### TRAIN RESULTS
- Image dice: 1: Left lung: 0.9842289238459231, Right lung: 0.9867282348039378, Bodymask: 0.9974271433421434
- Image housdorf: 1: Left lung: 17.832554500127006, Right lung: 16.401219466856727, Bodymask: 26.076809620810597
- Image dice: 7: Left lung: 0.9846273752537384, Right lung: 0.9751486850350803, Bodymask: 0.9985053543234377
- Image housdorf: 7: Left lung: 32.71085446759225, Right lung: 31.272991542223778, Bodymask: 20.0
- Image dice: 10: Left lung: 0.9874051320075206, Right lung: 0.986278954267094, Bodymask: 0.9973139561886235
- Image housdorf: 10: Left lung: 21.95449840010015, Right lung: 23.345235059857504, Bodymask: 9.0
- Image dice: 11: Left lung: 0.9802308407746525, Right lung: 0.9187749854050722, Bodymask: 0.9971283613561217
- Image housdorf: 11: Left lung: 39.761790704142086, Right lung: 34.14674215792775, Bodymask: 9.486832980505138
- Image dice: 12: Left lung: 0.9863789670462626, Right lung: 0.9887985807935349, Bodymask: 0.9984240218191143
- Image housdorf: 12: Left lung: 31.54362059117501, Right lung: 25.553864678361276, Bodymask: 5.0
- Image dice: 13: Left lung: 0.9886559266694869, Right lung: 0.9722513120119601, Bodymask: 0.9970907340384229
- Image housdorf: 13: Left lung: 22.293496809607955, Right lung: 35.07135583350036, Bodymask: 4.0
- Image dice: 14: Left lung: 0.9853585373045929, Right lung: 0.9847412793159765, Bodymask: 0.9976421511763593
- Image housdorf: 14: Left lung: 18.411952639521967, Right lung: 34.10278580995987, Bodymask: 6.0
